# Song Recommender Assignment

In [1]:
import graphlab

In [2]:
song_data = graphlab.SFrame('song_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to benuklove@gmail.com and will expire on December 18, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\Ben\AppData\Local\Temp\graphlab_server_1484535395.log.0


In [3]:
song_data.head()

user_id,song_id,listen_count,title,artist
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOAKIMP12A8C130995,1,The Cove,Jack Johnson
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Paco De Lucia
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBXHDL12A81C204C0,1,Stronger,Kanye West
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBYHAJ12A6701BF1D,1,Constellations,Jack Johnson
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODACBL12A8C13C273,1,Learn To Fly,Foo Fighters
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODDNQT12A6D4F5F7E,5,Apuesta Por El Rock 'N'Roll ...,Héroes del Silencio
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODXRTY12AB0180F3B,1,Paper Gangsta,Lady GaGa
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOFGUAY12AB017B0A8,1,Stacked Actors,Foo Fighters
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOFRQTD12A81C233C0,1,Sehr kosmisch,Harmonia
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOHQWYZ12A6D4FA701,1,Heaven's gonna burn youreyes ...,Thievery Corporationfeat. Emiliana Torrini ...


## 1. Counting unique users

### Find the number of unique users for specified artists

In [17]:
kw = song_data[song_data['artist'] == 'Kanye West']['user_id'].unique()
ff = song_data[song_data['artist'] == 'Foo Fighters']['user_id'].unique()
ts = song_data[song_data['artist'] == 'Taylor Swift']['user_id'].unique()
lg = song_data[song_data['artist'] == 'Lady GaGa']['user_id'].unique()

In [23]:
print "There are %d Kanye West listenters" % len(kw)
print "There are %d Foo Fighters listeners" % len(ff)
print "There are %d Taylor Swift listeners" % len(ts)
print "There are %d Lady GaGa listeners" % len(lg)

There are 2522 Kanye West listenters
There are 2055 Foo Fighters listeners
There are 3246 Taylor Swift listeners
There are 2928 Lady GaGa listeners


## 2. Using groupby-aggregate to find the most and least popular artist.

In [26]:
listen_totals = song_data.groupby(key_columns='artist', operations={'total_count': graphlab.aggregate.SUM('listen_count')})

In [41]:
# One way to find the most popular:
print listen_totals[listen_totals['total_count'] == listen_totals['total_count'].max()]['artist']

['Kings Of Leon', ... ]


In [42]:
# Another way; this gives the sorted table
listen_totals.sort(sort_columns='total_count', ascending=False)

artist,total_count
Kings Of Leon,43218
Dwight Yoakam,40619
Björk,38889
Coldplay,35362
Florence + The Machine,33387
Justin Bieber,29715
Alliance Ethnik,26689
OneRepublic,25754
Train,25402
The Black Keys,22184


In [43]:
# We can also find the least popular
listen_totals.sort(sort_columns='total_count')

artist,total_count
William Tabbert,14
Reel Feelings,24
Beyoncé feat. Bun B andSlim Thug ...,26
Diplo,30
Boggle Karaoke,30
harvey summers,31
Nâdiya,36
Jody Bernal,38
Aneta Langerova,38
Kanye West / Talib Kweli/ Q-Tip / Common / ...,38


## 3. (Optional) Basically combine above questions

### Using groupby-aggregate to find the most recommended songs

In [44]:
# Separate training and test data
train_data, test_data = song_data.random_split(.8, seed=0)

In [45]:
# Train a recommender model
personalized_model = graphlab.item_similarity_recommender.create(train_data, user_id='user_id', item_id='song')

Recsys training: model = item_similarity

Warning: Ignoring columns song_id, listen_count, title, artist;

To use one of these as a target column, set target = <column_name>

and use a method that allows the use of a target.

Preparing data set.

...

| 6.30s                               | 100              | 9952            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 7.6314s

In [46]:
# Select a portion of the 200,000 user entries
subset_test_users = test_data['user_id'].unique()[0:10000]

In [49]:
# Recommend one song for each of these 10k users
recommended_songs = personalized_model.recommend(subset_test_users, k=1)

recommendations finished on 1000/10000 queries. users per second: 1804.73

recommendations finished on 2000/10000 queries. users per second: 1754.12

recommendations finished on 3000/10000 queries. users per second: 1799.92

recommendations finished on 4000/10000 queries. users per second: 1824.56

recommendations finished on 5000/10000 queries. users per second: 1793.23

recommendations finished on 6000/10000 queries. users per second: 1784.21

recommendations finished on 7000/10000 queries. users per second: 1798.58

recommendations finished on 8000/10000 queries. users per second: 1812.2

recommendations finished on 9000/10000 queries. users per second: 1753.83

recommendations finished on 10000/10000 queries. users per second: 1679.76

In [50]:
recommended_songs

user_id,song,score,rank
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Cuando Pase El Temblor -Soda Stereo ...,0.0194504536115,1
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Where The Boat LeavesFrom (Album) - Zac Brown ...,0.0615360885859,1
f933855d675606737fdc191e9edff7625d08aae8 ...,Schießt die Deutschenraus - Mario Lang ...,0.0349425778669,1
4867d5516a280db13695b9b9c7ce6b574f34c6b4 ...,Two Steps_ Twice - Foals,0.0104654913857,1
968f1baebc490d3c6999ee6c85c5cab8b726b347 ...,Me_ Myself And I -Beyoncé ...,0.0166700103066,1
c067c22072a17d33310d7223d7b79f819e48cf42 ...,Grind With Me (ExplicitVersion) - Pretty Ricky ...,0.0459424376488,1
4454c4a192a467664286c8b58447709131ebdcf6 ...,Someday - Celtic Woman,0.0326410446848,1
2c494678b5582807c16390f13d076f785e20934e ...,What Country Is - LukeBryan ...,0.0264100432396,1
900ed526660df9c3442ebe736db94eb0bc251b73 ...,El Chupe Nibre (AlbumVersion) - Danger Doom ...,0.0100851539108,1
799816a69ec82a23a3a344971b485bc5417ec078 ...,Armour Love - La Roux,0.0593638803278,1


In [56]:
# Group by unique 'song' value and count the aggregation; store in new table
song_counts = recommended_songs.groupby(key_columns='song', operations={'count': graphlab.aggregate.COUNT()})

In [58]:
# Sort the table to find the most recommended song(s)
song_counts.sort(sort_columns='count', ascending=False)

song,count
Undo - Björk,426
Secrets - OneRepublic,399
Revelry - Kings Of Leon,229
You're The One - DwightYoakam ...,156
Hey_ Soul Sister - Train,113
Sehr kosmisch - Harmonia,113
Fireflies - CharttraxxKaraoke ...,102
Horn Concerto No. 4 in Eflat K495: II. Romance ...,91
OMG - Usher featuringwill.i.am ...,70
The Scientist - Coldplay,50
